In [19]:
import pystan
import arviz
import random

In [41]:
def do_exprinment(is_cheater):
    coin_result = flip_coin()
    if (coin_result):
        return is_cheater
    else:
        return flip_coin()

def flip_coin():
    result = False;
    rand_value = random.random();
    if (rand_value > 0.5):
        result = True;
    return result

def compute_result_to_list(num_of_cheaters, num_of_people):
    results = []
    all_population = range(num_of_people)
    cheaters_list = random.sample(all_population , k=num_of_cheaters)
    for i in all_population:
        result = False
        if i in cheaters_list:
            result = do_exprinment(True)
        else :
            result = do_exprinment(False)
        if (result):
            results.append(1)
        else:
            results.append(0)
            
    return results
    

In [34]:
generative_model = """

data {
    int<lower=0> N; // number of people answered the survey
    int<lower=0, upper = 1> y[N]; // boolean array of answers
}

parameters {
    real<lower=0, upper=1> theta; // the latent variable we want to infer
    real<lower=-1, upper=1> coin_results[N]; // helper coin results buffer
}


model {
    theta ~ beta(0.5, 0.5); // beta prior
    for (i in 1:N){
        coin_results[i] ~ normal(0 , 1);
        if (coin_results[i] >= 0){
            y[i] ~ bernoulli(theta);
        }
        else{
            y[i] ~ bernoulli(0.5);
        }
    }
    
}
"""

In [35]:
mixture_model = """

data {
    int<lower=0> N; // number of people answered the survey
    int<lower=0, upper = 1> y[N]; // boolean array of answers
}

parameters {
    real<lower=0, upper=1> theta; // the latent variable we want to infer
}


model {
    theta ~ beta(0.5, 0.5); // beta prior
    for (i in 1:N){
        target +=
            log_mix(0.5, bernoulli_lpmf(y[i] | theta), bernoulli_lpmf(y[i] | 0.5));
    }
    
}
"""

In [36]:
g_sm = pystan.StanModel(model_code=generative_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_10e97b055c992532f4f5189026ef2af5 NOW.


In [17]:
m_sm = pystan.StanModel(model_code=mixture_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1faba59f95facd8bf45fceea85f1353e NOW.


In [42]:
y1 = compute_result_to_list(15 , 100);
exp_1 = {
    'N': len(y1),
    'y': y1
}


y2 = compute_result_to_list(30 , 100);
exp_2 = {
    'N': len(y2),
    'y': y2
}


y3 = compute_result_to_list(45 , 100);
exp_3 = {
    'N': len(y3),
    'y': y3
}


y4 = compute_result_to_list(60, 100);
exp_4 = {
    'N': len(y4),
    'y': y4
}
y5 = compute_result_to_list(75, 100);
exp_5 = {
    'N': len(y5),
    'y': y5
}


y6 = compute_result_to_list(90, 100);
exp_6 = {
    'N': len(y6),
    'y': y6
}

In [45]:
g_fit = g_sm.sampling(data=exp_1, iter=1000, chains=4, control = {"adapt_delta" : 0.9 , 'max_treedepth': 11})

In [25]:
print(g_fit)

Inference for Stan model: anon_model_253b7b21ad7780ba0df8d0570d3407f2.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

                    mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta               0.15  5.1e-4   0.04   0.09   0.13   0.15   0.17   0.23   4771    1.0
coin_results[1]     0.45  4.5e-3   0.28   0.02   0.21   0.43    0.7   0.97   3970    1.0
coin_results[2]     0.46  4.4e-3   0.27   0.03   0.24   0.44   0.67   0.96   3674    1.0
coin_results[3]     0.47  4.2e-3   0.28   0.03   0.22   0.45   0.69   0.96   4397    1.0
coin_results[4]     0.46  4.4e-3   0.28   0.02   0.22   0.45   0.69   0.97   4114    1.0
coin_results[5]     0.47  4.0e-3   0.29   0.02   0.22   0.45   0.71   0.97   5182    1.0
coin_results[6]     0.46  4.5e-3   0.29   0.02   0.21   0.43    0.7   0.98   4161    1.0
coin_results[7]     0.45  4.7e-3   0.28   0.03   0.21   0.44   0.68   0.95   3541    1.0
coin_results

In [56]:
m_fit = m_sm.sampling(data=exp_1, iter=1000, chains=4, control = {"adapt_delta" : 0.9})

print(m_fit)

Inference for Stan model: anon_model_1faba59f95facd8bf45fceea85f1353e.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta   0.34  4.2e-3    0.1   0.14   0.26   0.34    0.4   0.54    583   1.01
lp__  -69.36    0.04   0.82 -71.66 -69.54 -69.05 -68.83 -68.77    386   1.02

Samples were drawn using NUTS at Sat Dec 28 15:31:41 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [47]:
m_fit2 = m_sm.sampling(data=exp_2, iter=1000, chains=4, control = {"adapt_delta" : 0.9})

print(m_fit2)

Inference for Stan model: anon_model_1faba59f95facd8bf45fceea85f1353e.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta   0.25  3.8e-3    0.1   0.08   0.19   0.25   0.32   0.44    627    1.0
lp__  -67.78    0.03   0.75 -69.95 -67.97 -67.49 -67.28 -67.22    506    1.0

Samples were drawn using NUTS at Sat Dec 28 15:27:45 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [57]:
m_fit3 = m_sm.sampling(data=exp_3, iter=5000, chains=7)

print(m_fit3)

Inference for Stan model: anon_model_1faba59f95facd8bf45fceea85f1353e.
7 chains, each with iter=5000; warmup=2500; thin=1; 
post-warmup draws per chain=2500, total post-warmup draws=17500.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta   0.21  1.4e-3    0.1   0.02   0.14    0.2   0.27    0.4   4612    1.0
lp__  -66.89    0.02   0.97 -69.81 -67.12 -66.51 -66.28 -66.21   2564    1.0

Samples were drawn using NUTS at Sat Dec 28 15:32:27 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [53]:
m_fit4 = m_sm.sampling(data=exp_4, iter=5000, chains=4, control = {"adapt_delta" : 0.9})
print(m_fit4)

Inference for Stan model: anon_model_1faba59f95facd8bf45fceea85f1353e.
4 chains, each with iter=5000; warmup=2500; thin=1; 
post-warmup draws per chain=2500, total post-warmup draws=10000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta   0.71  2.0e-3    0.1   0.51   0.64   0.71   0.78   0.91   2668    1.0
lp__  -68.69    0.02    0.9 -71.31 -68.87 -68.35 -68.14 -68.08   1742    1.0

Samples were drawn using NUTS at Sat Dec 28 15:30:06 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [55]:
m_fit5 = m_sm.sampling(data=exp_5, iter=5000, chains=4, control = {"adapt_delta" : 0.9})
print(m_fit5)

Inference for Stan model: anon_model_1faba59f95facd8bf45fceea85f1353e.
4 chains, each with iter=5000; warmup=2500; thin=1; 
post-warmup draws per chain=2500, total post-warmup draws=10000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta   0.82  2.1e-3    0.1   0.62   0.75   0.82   0.89   0.99   2133    1.0
lp__  -66.37    0.03   1.02 -69.42 -66.59 -65.96 -65.71 -65.65   1201    1.0

Samples were drawn using NUTS at Sat Dec 28 15:30:59 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).
